In [ ]:
import brainpy.respa as respa
import brainpy as bp
import brainpy.math as bm

a = respa.LIF(3200,V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto', V_initializer=bp.initialize.Normal(-55., 2.))

class MyNetwork(respa.Network):
  def __init__(self, *ds_tuple):
    super(MyNetwork, self).__init__(ds_tuple)
    self.a = respa.LIF(1000,V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto', V_initializer=bp.initialize.Normal(-55., 2.))
    self.b = respa.LIF(800,V_rest=-60., V_th=-50., V_reset=-60., tau=20., tau_ref=5., method='exp_auto', V_initializer=bp.initialize.Normal(-55., 2.))
    self.c = respa.Exponential(ds_tuple[0], self.a, bp.conn.FixedProb(0.02), g_max=10,tau=5., output=bp.synouts.COBA(E=0.), method='exp_auto')
    self.d = respa.Exponential(self.a[100:], self.b, bp.conn.FixedProb(0.02), g_max=10,tau=5., output=bp.synouts.COBA(E=0.), method='exp_auto')

net = MyNetwork(a)
net.build()
inputs = respa.input_transform([(net.a, 20)])
monitors = respa.monitor_transform([net.b], attr='spike')
runner = respa.DSRunner(
    net,
    monitors=monitors,
    inputs=inputs,
    jit=False
)
runner.run(10.)
if 'spike' in runner.mon:
  bp.visualize.raster_plot(runner.mon.ts, runner.mon['spike'], show=True)
  # print(net.pops_)
  # print(net.pops_split)
  # print(net.syns_)


In [ ]:
# print(net.nodes())
runner.run(10.)
bp.visualize.raster_plot(runner.mon.ts, runner.mon['spike'], show=True)

In [ ]:
class JansenRitModel(bp.dyn.DynamicalSystem):
  def __init__(self, num, C=135., method='exp_auto'):
    super(JansenRitModel, self).__init__()

    self.num = num

    # parameters #
    self.v_max = 5.  # maximum firing rate
    self.v0 = 6.  # firing threshold
    self.r = 0.56  # slope of the sigmoid
    # other parameters
    self.A = 3.25
    self.B = 22.
    self.a = 100.
    self.tau_e = 0.01  # second
    self.tau_i = 0.02  # second
    self.b = 50.
    self.e0 = 2.5
    # The connectivity constants
    self.C1 = C
    self.C2 = 0.8 * C
    self.C3 = 0.25 * C
    self.C4 = 0.25 * C

    # variables #
    # y0, y1 and y2 representing the firing rate of
    # pyramidal, excitatory and inhibitory neurones.
    self.y0 = bm.Variable(bm.zeros(self.num))
    self.y1 = bm.Variable(bm.zeros(self.num))
    self.y2 = bm.Variable(bm.zeros(self.num))
    self.y3 = bm.Variable(bm.zeros(self.num))
    self.y4 = bm.Variable(bm.zeros(self.num))
    self.y5 = bm.Variable(bm.zeros(self.num))
    self.p = bm.Variable(bm.ones(self.num) * 220.)

    # integral function
    self.derivative = bp.JointEq([self.dy0, self.dy1, self.dy2, self.dy3, self.dy4, self.dy5])
    self.integral = bp.odeint(self.derivative, method=method)

  def sigmoid(self, x):
    return self.v_max / (1. + bm.exp(self.r * (self.v0 - x)))

  def dy0(self, y0, t, y3): return y3

  def dy1(self, y1, t, y4): return y4

  def dy2(self, y2, t, y5): return y5

  def dy3(self, y3, t, y0, y1, y2):
    return (self.A * self.sigmoid(y1 - y2) - 2 * y3 - y0 / self.tau_e) / self.tau_e

  def dy4(self, y4, t, y0, y1, p):
    return (self.A * (p + self.C2 * self.sigmoid(self.C1 * y0)) - 2 * y4 - y1 / self.tau_e) / self.tau_e

  def dy5(self, y5, t, y0, y2):
    return (self.B * self.C4 * self.sigmoid(self.C3 * y0) - 2 * y5 - y2 / self.tau_i) / self.tau_i

  def update(self, tdi):
    t, dt = tdi['t'], tdi['dt']
    self.y0.value, self.y1.value, self.y2.value, self.y3.value, self.y4.value, self.y5.value = \
      self.integral(self.y0, self.y1, self.y2, self.y3, self.y4, self.y5, t, p=self.p, dt=dt)

jrm = JansenRitModel(num=6, C=bm.array([68., 128., 135., 270., 675., 1350.]))
dt = 0.1 / 1e3
duration=5.
all_ps = bm.random.uniform(120, 320, size=(int(duration / dt), 1))
runner = respa.DSRunner(
    jrm,
    monitors=['y0', 'y1', 'y2', 'y3', 'y4', 'y5','C1'],
    inputs=['p', all_ps, 'iter', '='],
    dt=dt
)
runner.run(duration)
start, end = int(2 / dt), int(duration / dt)
fig, gs = bp.visualize.get_figure(6, 3, 2, 3)
for i in range(6):
  fig.add_subplot(gs[i, 0])
  title = 'E' if i == 0 else None
  xlabel = 'time [s]' if i == 5 else None
  bp.visualize.line_plot(runner.mon.ts[start: end], runner.mon.y1[start: end, i],
                          title=title, xlabel=xlabel, ylabel='Hz')
  fig.add_subplot(gs[i, 1])
  title = 'P' if i == 0 else None
  bp.visualize.line_plot(runner.mon.ts[start: end], runner.mon.y0[start: end, i],
                          title=title, xlabel=xlabel)
  fig.add_subplot(gs[i, 2])
  title = 'I' if i == 0 else None
  bp.visualize.line_plot(runner.mon.ts[start: end], runner.mon.y2[start: end, i],
                          title=title, show=i == 5, xlabel=xlabel)